<a href="https://colab.research.google.com/github/alecbidaran/ML-Heart-disease-predictor/blob/main/ML_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fedesoriano/heart-failure-prediction

In [ ]:
!unzip heart-failure-prediction.zip

Archive:  heart-failure-prediction.zip
  inflating: heart.csv               


In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
df=pd.read_csv("heart.csv")
df


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelenc=LabelEncoder()
coloum_list=['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
x={}
for col in coloum_list:
    data=df[col].to_numpy()
    encode=labelenc.fit_transform(data.reshape(-1,1))
    x[col]=encode

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
encoded_x=pd.DataFrame.from_dict(x)
numerical_list=['Age', 'RestingBP', 'Cholesterol', 'FastingBS',
    'MaxHR', 'Oldpeak']
for col in numerical_list:
    encoded_x[col]=df[col]

In [ ]:
X=encoded_x[["Age","Sex","MaxHR","ChestPainType","ST_Slope","Cholesterol"]].to_numpy()
Y=df['HeartDisease'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=124)

In [ ]:
x_train=(x_train-x_train.min())/(x_train.max()-x_train.min())
x_test=(x_test-x_test.min())/(x_test.max()-x_test.min())

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dctree=DecisionTreeClassifier(max_depth=2)
dctree.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=2)

In [ ]:
pred=dctree.predict(x_test)
accuray=(pred==y_test)
accuray.mean()

0.8260869565217391

In [ ]:
encode_sex={name:i for i,name in enumerate(df["Sex"].unique())}
encode_chest={name:i for i,name in enumerate(df["ChestPainType"].unique())}
encode_slope={name:i for i, name in enumerate(df['ST_Slope'].unique())}
encode_slope

{'Up': 0, 'Flat': 1, 'Down': 2}

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz


--2022-12-07 11:52:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  17.4MB/s    in 0.8s    

2022-12-07 11:52:58 (17.4 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web i

In [ ]:
! ./ngrok authtoken 'add your ngrok token'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
app=Flask(__name__)
run_with_ngrok(app)
@app.route("/",methods=['GET'])
def greeting():
  return "Welcome to Garna Medical State"
@app.route("/Heart_disease",methods=["POST"])
def get_movies():
  json_object=request.json
  values=dict()
  values['Age']=json_object['Age']
  values['Sex']=encode_sex[json_object['Sex']]
  values['ChestPainType']=encode_chest[json_object['ChestPainType']]
  values["MaxHR"]=json_object['MaxHR']
  values["ST_Slope"]=encode_slope[json_object["ST_Slope"]]
  values['Cholestrol']=json_object['Cholestrol']
  encoded=pd.DataFrame.from_dict([values])
  x=encoded.to_numpy()
  x=x.reshape(1,-1)
  x=(x-x.min())/(x.max()-x.min())
  preds=dctree.predict_proba(x)
  print(encoded)
  #movie_list=recommend_movies(movies[0])
  results={'healthy':preds[0][0],'Diseased':preds[0][1]}
  print(results)
  return jsonify(results)



In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7d75-35-197-54-65.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [07/Dec/2022 12:58:39] "POST /Heart_disease HTTP/1.1" 200 -


   Age  Sex  ChestPainType  MaxHR  ST_Slope  Cholestrol
0   60    0              0     75         1         190
{'healthy': 0.8669064748201439, 'Diseased': 0.13309352517985612}


...

INFO:werkzeug:127.0.0.1 - - [07/Dec/2022 13:02:51] "POST /Heart_disease HTTP/1.1" 200 -


   Age  Sex  ChestPainType  MaxHR  ST_Slope  Cholestrol
0   40    0              0    101         1         170
{'healthy': 0.8669064748201439, 'Diseased': 0.13309352517985612}


INFO:werkzeug:127.0.0.1 - - [07/Dec/2022 13:03:45] "POST /Heart_disease HTTP/1.1" 200 -


   Age  Sex  ChestPainType  MaxHR  ST_Slope  Cholestrol
0   35    0              0     87         0         180
{'healthy': 0.3488372093023256, 'Diseased': 0.6511627906976745}


ERROR:__main__:Exception on /Heart_disease [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-104-7a4a66e56a93>", line 11, in get_movies
    values['Sex']=encode_sex[json_object['Sex']]
KeyError: 